In [208]:
import numpy as np

In [209]:
from scipy.optimize import least_squares

In [210]:
import pandas as pd

In [211]:
import matplotlib.pyplot as plt

In [212]:
import random

In [213]:
# CSVファイルの読み込み
csv_file_path = 'xy(3).csv'
data = pd.read_csv(csv_file_path)

# xとyのデータを抽出
x_data = data.iloc[:, 0].values  # CSVファイルの1列目をxとして使用
y_data = data.iloc[:, 1].values  # CSVファイルの2列目をyとして使用

In [214]:
x = np.linspace(0, 360, 4)

# 単位ベクトルの生成
def generate_unit_vectors():
    k = np.array(random.sample(range(1, 100), 3))
    n = k / np.linalg.norm(k)
    return n, generate_perpendicular_unit_vector(n)

# 磁場ベクトルの生成
def generate_perpendicular_unit_vector(n):
    if np.all(n == 0):
        raise ValueError("The input vector n should not be the zero vector.")
    
    if abs(n[0]) < abs(n[1]) and abs(n[0]) < abs(n[2]):
        w = np.array([1, 0, 0])
    elif abs(n[1]) < abs(n[0]) and abs(n[1]) < abs(n[2]):
        w = np.array([0, 1, 0])
    else:
        w = np.array([0, 0, 1])
    
    perp_vec = np.cross(n, w)
    return perp_vec / np.linalg.norm(perp_vec)

n, s = generate_unit_vectors()
c = np.cross(n, s)
theta = np.radians(x)

# ロドリゲスの回転公式
def rodrigues_rotation(axis, vec, theta):
    cos_theta = np.cos(theta)
    sin_theta = np.sin(theta)
    return (vec[:, np.newaxis] * cos_theta + 
            np.cross(axis[:, np.newaxis], vec[:, np.newaxis] * sin_theta, axis=0) + 
            axis[:, np.newaxis] * np.dot(axis, vec) * (1 - cos_theta))

h = rodrigues_rotation(n, s, theta)
hc = rodrigues_rotation(c, s, theta)

print(h)

[[-6.77475755e-01  6.16209231e-01  6.12665239e-02 -6.77475755e-01]
 [ 0.00000000e+00 -7.79548466e-01  7.79548466e-01  2.20471947e-16]
 [ 7.35545105e-01 -1.12206832e-01 -6.23338273e-01  7.35545105e-01]]


In [215]:
# h 行列を定義（例えば、3つのベクトルがある場合）
h = np.array([[1, 2, 3, 4],
              [5, 6, 7, 8],
              [9, 10, 11, 12]])

# h.T を表示
print("h.T:")
print(h.T)

# h.T の各列に対してループを回す
for h_vec in h.T:
    print("h_vec:")
    print(h_vec)
    print(h_vec.T)

h.T:
[[ 1  5  9]
 [ 2  6 10]
 [ 3  7 11]
 [ 4  8 12]]
h_vec:
[1 5 9]
[1 5 9]
h_vec:
[ 2  6 10]
[ 2  6 10]
h_vec:
[ 3  7 11]
[ 3  7 11]
h_vec:
[ 4  8 12]
[ 4  8 12]


In [216]:
# 回転行列
def rotation_matrix(kx, ky, kz, t, l):
    identity_matrix = np.identity(3)
    matrices = []
    for i in range(l):
        R = np.array([[0, -kz, ky], [kz, 0, -kx], [-ky, kx, 0]])
        theta = np.radians(t * i)
        rot_mat = identity_matrix + np.sin(theta) * R + (1 - np.cos(theta)) * np.dot(R, R)
        matrices.append(rot_mat)
    return matrices

kx = 0
ky = 0
kz = 1
l = 3
t = 120

R = rotation_matrix(kx, ky, kz, t, l)

h = np.array([[-6.11747641e-01, 3.91896056e-01, 2.19851586e-01, -6.11747641e-01], [0.00000000e+00, -8.59170967e-0, 8.59170967e-0, 2.42990788e-16], [7.91052984e-01, -3.29002630e-01, -4.62050354e-01, 7.91052984e-01]])
K = np.array([[0.1, 0.1, 0.1], [0.1, 0.1, 0.1], [0.1, 0.1, 0.1]])

# 組み合わせて考えた関数
def calculate_knight_shift(R, h, hc, K):
    h_vec = np.array([h[0][j], h[1][j], h[2][j]])
    hc_vec = np.array([hc[0][j], hc[1][j], hc[2][j]])

    K_rot_values = [np.dot(h_vec.T, np.dot(R, np.dot(K, np.dot(np.linalg.inv(R), h_vec))))
                    for j in range(len(h[0]))]
    Kc_rot_values = [np.dot(hc_vec.T, np.dot(R, np.dot(K, np.dot(np.linalg.inv(R), hc_vec))))
                    for j in range(len(hc[0]))]
    return K_rot_values, Kc_rot_values

print(h.T)
print(calculate_knight_shift(R, h, hc, K))

[[-6.11747641e-01  0.00000000e+00  7.91052984e-01]
 [ 3.91896056e-01 -8.59170967e+00 -3.29002630e-01]
 [ 2.19851586e-01  8.59170967e+00 -4.62050354e-01]
 [-6.11747641e-01  2.42990788e-16  7.91052984e-01]]


NameError: name 'j' is not defined